In [1]:
import asyncio
import aiomysql

loop = asyncio.get_event_loop()

In [2]:
DATABASE_CONFIG = {
    'host': 'localhost',
    'port': 3306,
    'db': 'test',
    'user': 'root',
    'password': '*********',
    'charset':'utf8',
    'autocommit': True
}

In [3]:
"""
DATABASE_USER = 'root'
PASSWORD = '**********'
DATABASE_NAME = 'test'
HOST = 'localhost'
PORT = 3306
"""

DATABASE_CONFIG = {
    'host': 'localhost',
    'port': 3306,
    'db': 'test',
    'user': 'root',
    'password': 'Mysql@zhangqi',
    'charset':'utf8',
    'autocommit': True
}

async def go(loop):
    try:
        pool = await aiomysql.create_pool(**DATABASE_CONFIG,loop=loop)

        async with pool.acquire() as conn:
            async with conn.cursor() as cur:
                await cur.execute("SELECT * FROM test")
                # await cur.execute("INSERT INTO test(name, password) VALUES ('liberqi','qwe123')")
                print(cur.description)
                rs = await cur.fetchall()
                
    finally:
        pool.close()
        await pool.wait_closed()
        print(cur.closed)
        return rs

In [4]:
rs = loop.run_until_complete(go(loop))

(('id', 3, None, 11, 11, 0, False), ('name', 253, None, 50, 50, 0, False), ('password', 253, None, 50, 50, 0, False))
True


In [5]:
rs

((1, 'tom', '123456'),
 (2, 'marry', '145867'),
 (3, 'jim', 'awe123'),
 (4, 'liberqi', 'qwe123'),
 (6, 'harry', 'asfdvg'),
 (7, 'William', '147159'),
 (8, 'Bobby', '666666'))

In [6]:
"""全局连接池,连接池中直接获取数据库连接
所有数据存取完毕时关闭数据库连接
"""
import asyncio
import aiomysql

class AsyncDatabase:

    def __init__(self, loop=None, is_pool=False):
        """Database operations：mysql use aiomysql 
        param is_pool: create conntection pool default False
        """
        self.loop = loop or asyncio.get_event_loop()
        self.is_pool = is_pool # 是否建立连接池,默认False
        
    # also can use asyncpg
    
    # create conntection pool
    async def create_pool(self):
         
        self.conn_pool = await aiomysql.create_pool(**DATABASE_CONFIG,loop=self.loop)

        
    # get conntection   
    async def get_conn(self):
        if not self.is_pool:
            self.conn = await aiomysql.connect(**DATABASE_CONFIG,loop=self.loop)
        else:
            self.conn = await self.conn_pool.acquire()
        
    # close conntection
    async def close(self):
        """close db connect
        """
        if not self.is_pool:
            self.conn.close()
        else:
            self.conn.close()
            self.conn_pool.close()
            await self.conn_pool.wait_closed()
        
    # select data
    async def select_data(self, sql, size=None):
        async with self.conn.cursor() as cursor:
        # cur = await conn.cursor(aiomysql.DictCursor)
            try:
                await cursor.execute(sql)
                if size:
                    rows = await cursor.fetchmany(size)
                else:
                    rows = await cursor.fetchall()
            except aiomysql.MySQLError as e:
                await self.close()
                raise e
            return rows
    
    # insert data
    async def insert_data(self, sql, data):
        async with self.conn.cursor() as cursor:
            try:
                await cursor.executemany(sql, data)
                # await cursor.executemany(sql, data)
            except aiomysql.MySQLError as e:
                await self.close()
                raise e



In [7]:
# AsyncDatabase test select
async def select(loop):
    try:
        dber = AsyncDatabase(loop)
        await dber.get_conn()
        sql = 'SELECT * FROM test'
        data = await dber.select_data(sql)
    finally:
        await dber.close()
    return data

# test select
if __name__ == '__main__':
    data = loop.run_until_complete(select(loop))
    print(data)

((1, 'tom', '123456'), (2, 'marry', '145867'), (3, 'jim', 'awe123'), (4, 'liberqi', 'qwe123'), (6, 'harry', 'asfdvg'), (7, 'William', '147159'), (8, 'Bobby', '666666'))


In [8]:
# AsyncDatabase test insert
name = ['harry', 'William', 'Bobby']
password = ['asfdvg', '147159', '666666']
data = [name, password]
async def insert(loop):
    try:
        
        dber = AsyncDatabase(loop)
        await dber.get_conn()
        sql = 'INSERT INTO test(name, password) VALUES (%s, %s)'
        await dber.insert_data(sql,zip(name,password))
    finally:
        await dber.close()
        print('done!')
        
if __name__ == '__main__':        
    loop.run_until_complete(insert(loop))

done!


In [ ]:
SQL_INSTRUCTION = """INSERT INTO test(href, title, compound, layout,gross_floor_area, 
                    distribute, road, floor,orientation, rent_per_month, added_at,
                    total_views, subway_line, subway_station,subway_distance, number, 
                    address,latest_week_views, room_type, img_url,captured_at) 
                    VALUES (%s, %s, %s, %s, %s,%s, %s, %s, %s, %s,%s, 
                    %s, %s, %s, %s,%s, %s, %s, %s, %s,%s, %s)"""